<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/tpu/fine_tune_tpu_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune Xception TPUs (Colab)&nbsp;<img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50">

### Enabling and testing the TPU

In [10]:
import re
import numpy as np
import tensorflow as tf

from matplotlib import pyplot as plt

tf.__version__

'2.3.0'

In [11]:
#@title Enable TPU
use_tpu = True #@param ["False", "True"] {type:"raw"}

if use_tpu:
  try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tpu_spec = tpu.cluster_spec().as_dict()['worker']
    print('Running on TPU:', tpu_spec)
  except ValueError:
    print('ERROR: Not connected to a TPU.')

  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Running on TPU: ['10.27.206.210:8470']


INFO:tensorflow:Initializing the TPU system: grpc://10.27.206.210:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.27.206.210:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)



## Input data

Our input data is stored on Google Cloud Storage. To more fully use the parallelism TPUs offer us, and to avoid bottlenecking on data transfer, we've stored our input data in TFRecord files, 230 images per file.

Below, we make heavy use of `tf.data.experimental.AUTOTUNE` to optimize different parts of input loading.

All of these techniques are a bit overkill for our (small) dataset, but demonstrate best practices for using TPUs.

In [13]:
AUTO = tf.data.experimental.AUTOTUNE
IMAGE_SIZE = [331, 331]
GCS_PATTERN = 'gs://flowers-public/tfrecords-jpeg-331x331/*.tfrec'
VALIDATION_SPLIT = 0.19
BATCH_SIZE = 16 * tpu_strategy.num_replicas_in_sync

print('Batch Size:', BATCH_SIZE)

Batch Size: 128


In [22]:
filenames = tf.io.gfile.glob(GCS_PATTERN)
split = len(filenames) - int(len(filenames) * VALIDATION_SPLIT)
train_fns = filenames[:split]
validation_fns = filenames[split:]
print('Train len:', len(train_fns), ' ', 'Validation len:', len(validation_fns))

Train len: 13   Validation len: 3


In [19]:
len(filenames)

16